# Copernicus Data Space Ecosystem

The CDSE is an open ecosystem that provides free and instant access to a wide range of data and services from the Copernicus Sentinel missions and more on our planet’s land, oceans and atmosphere including Earth observation data from the Copernicus Sentinel satellites.

## Sentinel Hub
The Sentinel Hub is a set of RESTful APIs that provide access to various satellite imagery archives. It allows for accessing raw satellite data, rendered images, statistical analysis, and other features.

## Sentinel-2

The Copernicus Sentinel-2 mission consists of two polar-orbiting satellites that are positioned in the same sun-synchronous orbit, with a phase difference of 180°.

It aims to monitor changes in land surface conditions. The satellites carry a multispectral imager with a wide swath width (290 km) and have a high revisit time (5 days). The imager provides a versatile set of 13 spectral bands spanning from the visible and near infrared to the shortwave infrared, featuring four spectral bands at 10 m, six bands at 20 m and three bands at 60 m spatial resolution.

### Level 1C Data

Sentinel-2 Level 1C products are available globally from 2015 onwards. These products are resampled with a constant Ground Sampling Distance (GSD) of 10, 20 and 60 m, depending on the native resolution of the different spectral bands. Pixel coordinates refer to the upper left corner of the pixel.

<img src="data/documentation/sen2_bands.png" width=600 height=300>

## Sentinel-3

The main objective of Sentinel-3 is to measure sea surface topography, sea and land surface temperature, and ocean and land surface colour with high accuracy and reliability to support ocean forecasting systems, environmental monitoring and climate monitoring. Sentinel-3 observations also support applications based on vegetation as well as fire, inland waters (river and lake water surface height), the cryosphere (i.e., land ice and sea-ice thickness) and atmosphere.

<img src="data/documentation/sen3_bands.png" width=600 height=300>

### Sea and Land Surface Temperature Radiometer (SLSTR) Data

SLSTR is based on ENVISAT's Advanced Along Track Scanning Radiometer (AATSR), to determine global Sea Surface Temperatures to an accuracy of better than 0.3 K. The SLSTR improves the along track scanning dual-view technique of AATSR and provides advanced atmospheric correction. SLSTR measures in nine spectral channels and two additional bands. A 1.3 micron band aimed at cirrus detection and a 2 micron band aimed at vegetation, mineral and atmospheric corrections for very turbid waters. An SLSTR with spatial resolution in the visible and shortwave infra-red channels of 500 m and 1 km in the thermal infra-red channels.

<img src="data/documentation/sen3_bands_classes.png" width=600 height=300>

## Sentinel Hub APIs of Interest

### Statistical API
The Statistical API allows for obtaining statistics that are calculated based on satellite imagery, without the requirement of downloading images. When you send a request to the Statistical API, you can define your area of interest, time frame, evalscript, and the statistical measures that need to be calculated. The result is a json file with the requested set of statistics for your area of interest.

### Processing API
The Processing API is the most frequently used API in Sentinel Hub. It generates images using satellite data for a user-specified area of interest, time range, processing, and visualization. This API offers the easiest way to process satellite data, generate personalized visual representations of satellite images and maps using processed data, or download raw data.

**More info:**
- [Sentinel Hub](https://dataspace.copernicus.eu/analyse/apis/sentinel-hub)
- [Sentinel Missions](https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data)
- [Sentinel-2](https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data/sentinel-2)
- [Sentinel-2 Level 1C](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel2.html#sentinel-2-level-1c-top-of-atmosphere-toa)
- [Sentinel-2 Level 1C - API access, requests](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S2L1C.html)
- [Complete list of Sentinel-2 composites and algorithms](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel/sentinel-2/)
- [Sentinel-3](https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data/sentinel-3)
- [Sentinel-3 SLSTR](https://sentiwiki.copernicus.eu/web/s3-slstr-instrument)
- [Interactive Map with Sentinel-2 L1C data](https://browser.dataspace.copernicus.eu/?zoom=11&lat=15.50919&lng=32.49491&themeId=DEFAULT-THEME&visualizationUrl=U2FsdGVkX19thDCcH4aD62EybqrdKXcd17vDo0WcFB6ko02GLJfB8SIQCq9sqQv7S2j2YALE4BYcee7XZX18YmeVrBEZNebFhpic1tporv2XUjNN94K80mCTRL9%2FFUba&datasetId=S2_L1C_CDAS&fromTime=2025-01-01T00%3A00%3A00.000Z&toTime=2025-01-01T23%3A59%3A59.999Z&layerId=7-NDWI&demSource3D=%22MAPZEN%22&cloudCoverage=10&dateMode=SINGLE)

In [771]:
import json
import time
import requests
import geopandas as gpd
from tqdm import tqdm
import shapely
from shapely.geometry import shape, Polygon, MultiPolygon
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from getpass import getpass

In [310]:
tqdm.pandas()

## Creating an OAuth client

In [775]:
client_id = getpass.getpass("CDSE Client ID: ")
client_secret = getpass.getpass("CDSE Client secret: ")

CDSE Client ID:  ········
CDSE Client secret:  ········


In [642]:
# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

In [644]:
# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

In [645]:
# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


## Loading Geographic Data of Administrative Boundaries

Sentinel API uses WGS84 (latitude-longitude) coordinates, which corresponds to `EPSG:4326`, GADM uses `urn:ogc:def:crs:OGC:1.3:CRS84` which is functionally equivalent to EPSG:4326. Both represent the WGS84 geographic coordinate system, which uses latitude and longitude coordinates.

However, some tools or systems might not recognize `urn:ogc:def:crs:OGC:1.3:CRS84` directly as `EPSG:4326`. To avoid potential compatibility issues, it's best to explicitly convert the CRS to `EPSG:4326`.

Data source: [GADM](https://gadm.org/download_country.html)

In [196]:
def ensure_epsg4326(file_path):
    """
    Ensure that the GeoDataFrame is in EPSG:4326, converting if necessary.
    
    Parameters:
        file_path (str): Path to the GeoJSON file.
        
    Returns:
        gpd.GeoDataFrame: GeoDataFrame with geometries in EPSG:4326.
    """
    gdf = gpd.read_file(file_path)
    
    current_crs = gdf.crs
    print(f"Current CRS: {current_crs}")
    
    if current_crs != "EPSG:4326":
        print("Reprojecting to EPSG:4326...")
        gdf = gdf.to_crs("EPSG:4326")
    else:
        print("CRS is already EPSG:4326.")
    
    return gdf

In [204]:
geojson_files = {
    "state": "data/administrative_boundaries/gadm/gadm41_SDN_1.json",
    "district": "data/administrative_boundaries/gadm/gadm41_SDN_2.json",
    "subdivision": "data/administrative_boundaries/gadm/gadm41_SDN_3.json"
}

In [300]:
for file in geojson_files:
    converted_gdf = ensure_epsg4326(geojson_files[file])
    converted_gdf.to_file(f"data/administrative_boundaries/gadm/gadm41_SDN_{list(geojson_files.keys()).index(file)+1}.geojson", driver="GeoJSON")

Current CRS: EPSG:4326
CRS is already EPSG:4326.
Current CRS: EPSG:4326
CRS is already EPSG:4326.
Current CRS: EPSG:4326
CRS is already EPSG:4326.


In [285]:
def convert_coordinates_format(geometry):
    """
    Convert coordinates from GeoDataFrame geometry (tuple format) to the required format.
    
    Parameters:
        geometry (shapely.geometry): The geometry object (Polygon or MultiPolygon).
        
    Returns:
        list: Coordinates in the required format, i.e., a list of lists of [longitude, latitude].
    """
    if geometry.geom_type == 'MultiPolygon':
        # convert each Polygon within the MultiPolygon to the required format
        return [
            [list(polygon.exterior.coords) for polygon in geometry.geoms]
        ]
    elif geometry.geom_type == 'Polygon':
        # convert the Polygon to the required format
        return [list(geometry.exterior.coords)]
    else:
        return []

In [287]:
def load_and_process_gadm(geojson_files):
    """
    Load and process GADM GeoJSON files for different administrative levels.
    
    Parameters:
        geojson_files (dict): Dictionary with keys as levels (e.g., "state", "district", "subdivision")
                              and values as paths to the corresponding GeoJSON files.
                              
    Returns:
        dict: A nested dictionary with levels as keys and their processed data.
    """
    processed_data = {}

    for level, file_path in geojson_files.items():
        gdf = gpd.read_file(file_path)
        
        level_data = []
        for _, row in gdf.iterrows():
            level_data.append({
                "id": row.get("GID_1") or row.get("GID_2") or row.get("GID_3"),
                "name": row.get("NAME_1") or row.get("NAME_2") or row.get("NAME_3"),
                "type": row.get("TYPE_1") or row.get("TYPE_2") or row.get("TYPE_3"),
                "geometry": convert_coordinates_format(row.get("geometry"))
            })
        
        processed_data[level] = level_data

    return processed_data

In [289]:
processed_gadm_data = load_and_process_gadm(geojson_files)

In [293]:
processed_gadm_data.keys()

dict_keys(['state', 'district', 'subdivision'])

In [302]:
for level in list(processed_gadm_data.keys()):
    with open(f"data/administrative_boundaries/gadm/gadm41_SDN_{list(processed_gadm_data.keys()).index(level)+1}_coordinates_extracted.json", "w") as outfile:
        json.dump(processed_gadm_data[level], outfile, indent=2)

### Extract coordinates example on the district subdivisions data in gdf

In [438]:
gdf = gpd.read_file(geojson_files["subdivision"])

In [447]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   GID_3        237 non-null    object  
 1   GID_0        237 non-null    object  
 2   COUNTRY      237 non-null    object  
 3   GID_1        237 non-null    object  
 4   NAME_1       237 non-null    object  
 5   NL_NAME_1    237 non-null    object  
 6   GID_2        237 non-null    object  
 7   NAME_2       237 non-null    object  
 8   NL_NAME_2    237 non-null    object  
 9   NAME_3       237 non-null    object  
 10  VARNAME_3    237 non-null    object  
 11  NL_NAME_3    237 non-null    object  
 12  TYPE_3       237 non-null    object  
 13  ENGTYPE_3    237 non-null    object  
 14  CC_3         237 non-null    object  
 15  HASC_3       237 non-null    object  
 16  geometry     237 non-null    geometry
 17  coordinates  237 non-null    object  
dtypes: geometry(1), object

In [445]:
gdf

,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry,coordinates
0,SDN.1.1.1_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElKamlin,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...","[[[(32.9784, 15.1733), (33.0415, 15.2452), (33..."
1,SDN.1.1.2_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElMasid,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((33.0415 15.2452, 32.9784 15.17...","[[[(33.0415, 15.2452), (32.9784, 15.1733), (32..."
2,SDN.1.1.3_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElMiEiliq,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.9784 15.1733, 33.2035 14.88...","[[[(32.9784, 15.1733), (33.2035, 14.8855), (33..."
3,SDN.1.1.4_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,EsSideira,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.5843 15.2263, 32.6041 15.22...","[[[(32.5843, 15.2263), (32.6041, 15.2208), (32..."
4,SDN.1.2.1_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.2_1,AlMahagil,NA,AlAzazi,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.6635 14.3983, 32.6733 14.44...","[[[(32.6635, 14.3983), (32.6733, 14.4499), (32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,SDN.18.4.3_1,SDN,Sudan,SDN.18_1,WhiteNile,النيلالأبيض‎,SDN.18.4_1,Kosti,NA,ErRawat,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.4448 11.9057, 32.1744 11.90...","[[[(32.4448, 11.9057), (32.1744, 11.9054), (32..."
233,SDN.18.4.4_1,SDN,Sudan,SDN.18_1,WhiteNile,النيلالأبيض‎,SDN.18.4_1,Kosti,NA,EzZeleit,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.7516 12.9151, 32.751 12.908...","[[[(32.7516, 12.9151), (32.751, 12.908), (32.7..."
234,SDN.18.4.5_1,SDN,Sudan,SDN.18_1,WhiteNile,النيلالأبيض‎,SDN.18.4_1,Kosti,NA,KostiGenoub,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.6045 13.3092, 32.609 13.29,...","[[[(32.6045, 13.3092), (32.609, 13.29), (32.62..."
235,SDN.18.4.6_1,SDN,Sudan,SDN.18_1,WhiteNile,النيلالأبيض‎,SDN.18.4_1,Kosti,NA,KostiShmal,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.3663 13.6897, 32.3742 13.68...","[[[(32.3663, 13.6897), (32.3742, 13.6853), (32..."


In [440]:
gdf["coordinates"] = gdf["geometry"].progress_apply(lambda x: convert_coordinates_format(x))

100%|██████████| 237/237 [00:00<00:00, 7638.55it/s]


## Extract Composites from Sentinel-2 Data via Sentinel Hub's Statistical API

### Normalized Difference Vegetation Index (NDVI)

NDVI is a simple, but effective index for quantifying green vegetation. It normalizes green leaf scattering in Near Infra-red wavelengths with chlorophyll absorption in red wavelengths.

The value range of the NDVI is -1 to 1. Negative values of NDVI (values approaching -1) correspond to water. Values close to zero (-0.1 to 0.1) generally correspond to barren areas of rock, sand, or snow. Low, positive values represent shrub and grassland (approximately 0.2 to 0.4), while high values indicate temperate and tropical rainforests (values approaching 1). It is a good proxy for live green vegetation.

The normalized difference vegetation index, abbreviated NDVI, is defined as:

<img src="data/documentation/ndvi.png" width="300" height="150">

For Sentinel-2, the index looks like this:

<img src="data/documentation/ndvi_sen2.png" width="300" height="150">


### Normalized Difference Water Index (NDWI)

NDWI is used to monitor changes related to water content in water bodies. As water bodies strongly absorb light in visible to infrared electromagnetic spectrum, NDWI uses green and near infrared bands to highlight water bodies. It is sensitive to built-up land and can result in over-estimation of water bodies. The index was proposed by McFeeters, 1996.

Sentinel-2 NDWI = (B03 - B08) / (B03 + B08)

Index values greater than 0.5 usually correspond to water bodies. Vegetation usually corresponds to much smaller values and built-up areas to values between zero and 0.2.

NDWI index is often used synonymously with the NDMI index, often using NIR-SWIR combination as one of the two options. NDMI seems to be consistently described using NIR-SWIR combination. As the indices with these two combinations work very differently, with NIR-SWIR highlighting differences in water content of leaves, and GREEN-NIR highlighting differences in water content of water bodies, the indices in the repository are separated as NDMI using NIR-SWIR, and NDWI using GREEN-NIR.

### Land Surface Temperature (LST)

Land Surface Temperature (LST) is the radiative skin temperature of the land derived from infrared radiation. In the SLSTR project, "skin" temperature refers to the temperature of the top surface when in bare soil conditions, and to the effective emitting temperature of vegetation "canopies" as determined from a view of the top of a canopy.

A simplified definition would be how hot the "surface" of the Earth would feel to the touch in a particular location. From a satellite's point of view, the "surface" is whatever it sees when it looks through the atmosphere to the ground. It could be snow and ice, the grass on a lawn, the roof of a building or the leaves in the canopy of a forest. LST is not the same as the air temperature that is included in the daily weather report.

In [636]:
def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

In [658]:
sentinel_data_type_ids = {"sentinel_2": "sentinel-2-l1c", "sentinel_3": "sentinel-3-slstr"}

In [660]:
evalscripts = {
    "NDVI": """
    //VERSION=3
    function setup() {
        return {
            input: ["B04", "B08", "dataMask"],
            output: [{ id: "data", bands: 1 }, { id: "dataMask", bands: 1 }]
        };
    }
    function evaluatePixel(sample) {
        let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
        return { data: [ndvi], dataMask: [sample.dataMask] };
    }
    """,
    "NDWI": """
    //VERSION=3
    function setup() {
        return {
            input: ["B03", "B08", "dataMask"],
            output: [{ id: "data", bands: 1 }, { id: "dataMask", bands: 1 }]
        };
    }
    function evaluatePixel(sample) {
        let ndwi = (sample.B03 - sample.B08) / (sample.B03 + sample.B08);
        return { data: [ndwi], dataMask: [sample.dataMask] };
    }
    """
}

In [662]:
def get_sentinel_statistics(oauth_session, data_type_id, region_coordinates, 
        resx, resy, start_date="2015-07-01", end_date=None, 
        metrics=["NDVI", "NDWI"], granularity="P1D"):
    """
    Retrieve NDVI, NDWI and LST metrics for a specified region and time range using SentinelHub Statistical API.
    
    Parameters:
    - oauth_session (OAuth2Session): Authenticated session with the SentinelHub API.
    - data_type_id (str): Id of Sentinel mission and product to get data from.
    - region_coordinates (list): GeoJSON geometry defining the region of interest.
    - start_date (str): Start date in ISO 8601 format (default: "2015-07-01").
    - end_date (str): End date in ISO 8601 format (default: today's date if None).
    - metrics (list): List of metrics to retrieve (e.g., ["NDVI", "NDWI"]).
    - granularity (str): Temporal granularity (e.g., "P1D" for daily, "P1M" for monthly).
    
    Returns:
    - dict: Dictionary of metrics and their statistical outputs.
    """
    if end_date is None:
        from datetime import date
        end_date = date.today().isoformat()
    
    url = "https://sh.dataspace.copernicus.eu/api/v1/statistics"
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    results = {}

    for metric in metrics:
        if metric not in evalscripts:
            raise ValueError(f"Unsupported metric: {metric}")

        stats_request = {
            "input": {
                "bounds": {
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": region_coordinates
                    },
                    "properties": {
                        "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
                    }
                },
                "data": [{
                    "type": data_type_id,
                    "dataFilter": {"mosaickingOrder": "leastCC"}
                }]
            },
            "aggregation": {
                "timeRange": {"from": start_date + "T00:00:00Z", "to": end_date + "T23:59:59Z"},
                "aggregationInterval": {"of": granularity},
                "evalscript": evalscripts[metric],
                "resx": resx,
                "resy": resy
            }
        }

        response = oauth_session.post(url, headers=headers, json=stats_request)
        if response.status_code != 200:
            raise ValueError(f"Failed to retrieve data for {metric}: {response.text}")
        
        results[metric] = response.json()

    return results

In [686]:
metrics = list()

In [688]:
for i in tqdm(range(0,gdf.shape[0])):
    stats = get_sentinel_statistics(
        oauth_session=oauth,
        data_type_id = sentinel_data_type_ids["sentinel_2"],
        region_coordinates=gdf["coordinates"][i][0],
        resx=10, resy=10,
        start_date="2015-07-01",
        metrics=["NDVI", "NDWI"],
        granularity="P1D"
    )
    metrics.append(stats)

    time.sleep(2)
    
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                client_secret=client_secret, include_client_id=True)

    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

100%|██████████| 236/236 [1:10:04<00:00, 17.82s/it]


In [738]:
ndvi = list()
ndwi = list()

In [739]:
for item in tqdm(metrics):
    ndvi.append(item["NDVI"])
    ndwi.append(item["NDWI"])

100%|██████████| 237/237 [00:00<00:00, 620505.65it/s]


In [741]:
gdf["ndvi"] = ndvi
gdf["ndwi"] = ndwi

In [744]:
gdf.head()

,GID_3,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,NL_NAME_2,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry,coordinates,ndvi,ndwi
0,SDN.1.1.1_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElKamlin,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...","[[[(32.9784, 15.1733), (33.0415, 15.2452), (33...",{'data': [{'interval': {'from': '2015-10-20T00...,{'data': [{'interval': {'from': '2015-10-20T00...
1,SDN.1.1.2_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElMasid,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((33.0415 15.2452, 32.9784 15.17...","[[[(33.0415, 15.2452), (32.9784, 15.1733), (32...",{'data': [{'interval': {'from': '2015-10-20T00...,{'data': [{'interval': {'from': '2015-10-20T00...
2,SDN.1.1.3_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,ElMiEiliq,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.9784 15.1733, 33.2035 14.88...","[[[(32.9784, 15.1733), (33.2035, 14.8855), (33...",{'data': [{'interval': {'from': '2015-10-20T00...,{'data': [{'interval': {'from': '2015-10-20T00...
3,SDN.1.1.4_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.1_1,AlKamlin,NA,EsSideira,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.5843 15.2263, 32.6041 15.22...","[[[(32.5843, 15.2263), (32.6041, 15.2208), (32...",{'data': [{'interval': {'from': '2015-10-20T00...,{'data': [{'interval': {'from': '2015-10-20T00...
4,SDN.1.2.1_1,SDN,Sudan,SDN.1_1,AlJazirah,ولايةالجزيرة,SDN.1.2_1,AlMahagil,NA,AlAzazi,NA,NA,Unknown,Unknown,NA,NA,"MULTIPOLYGON (((32.6635 14.3983, 32.6733 14.44...","[[[(32.6635, 14.3983), (32.6733, 14.4499), (32...",{'data': [{'interval': {'from': '2015-10-20T00...,{'data': [{'interval': {'from': '2015-10-20T00...


## Saving the Data

In [746]:
# as a GeoJSON file
gdf.to_file("data/sentinel_2/ndvi_ndwi_for_sdn_by_district_subdivision.geojson", driver="GeoJSON")

# as a Shapefile
gdf.to_file("data/sentinel_2/ndvi_ndwi_for_sdn_by_district_subdivision.shp", driver="ESRI Shapefile")

/var/folders/g8/tc9k918x5hncpxzn40y8c00h0000gn/T/ipykernel_22670/2002122221.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("data/sentinel_2/ndvi_ndwi_for_sdn_by_district_subdivision.shp", driver="ESRI Shapefile")
/opt/anaconda3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'coordinates' to 'coordinate'
  ogr_write(
/opt/anaconda3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value '[[[(32.9784, 15.1733), (33.0415, 15.2452), (33.046, 15.243), (33.071, 15.239), (33.079, 15.234), (33.086, 15.213), (33.091, 15.204), (33.127, 15.157), (33.142, 15.144), (33.162, 15.136), (33.173, 15.126), (33.179, 15.118), (33.182, 15.108), (33.184, 15.068), (33.186, 15.058), (33.192, 15.05), (33.216, 15.031), (33.221, 15.023), (33.2228, 15.0123), (33.222, 14.998), (33.213, 14.979), (33.211, 14.967), (33.211, 14.955), (33.213, 14.947), (33.218, 14.938), (33.219, 14.9

In [756]:
# to JSON (without geometry data)
gdf.drop(columns='geometry').to_json("data/sentinel_2/ndvi_ndwi_for_sdn_by_district_subdivision.json", orient="records")

In [752]:
# to csv (without geometry data)
# GeoDataFrame's geometry will be converted to WKT (Well-Known Text) format
gdf.drop(columns='geometry').to_csv("data/sentinel_2/ndvi_ndwi_for_sdn_by_district_subdivision.csv", index=False)